# <font color = #7B68EE> ROSSMANN Sales Store Prediction </font>

- Project REPO [ROSSMANN Sales Store Prediction](https://github.com/pmusachio/rossmann_store_sales_prediction)
- **CFO needs to plan store renovations, and to decide how much to invest in each one, he needs the daily sales forecast per store for the next 6 weeks**

# <font color = #7B68EE> 0.0 Imports </font>

In [ ]:
# import math
# import numpy  as np
# import pandas as pd
# import random
# import pickle
# import requests
# import warnings
# import inflection
# import seaborn as sns
# import xgboost as xgb

# from scipy                 import stats  as ss
# from boruta                import BorutaPy
# from matplotlib            import pyplot as plt
# from IPython.display       import Image
# from IPython.core.display  import HTML


# from sklearn.metrics       import mean_absolute_error, mean_squared_error
# from sklearn.ensemble      import RandomForestRegressor
# from sklearn.linear_model  import LinearRegression, Lasso
# from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder

# warnings.filterwarnings( 'ignore' )

In [1]:
import numpy  as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

## 0.1 Helper Functions

In [ ]:
# def cross_validation( x_training, kfold, model_name, model, verbose=False ):
#     mae_list = []
#     mape_list = []
#     rmse_list = []
#     for k in reversed( range( 1, kfold+1 ) ):
#         if verbose:
#             print( '\nKFold Number: {}'.format( k ) )
#         # start and end date for validation 
#         validation_start_date = x_training['date'].max() - datetime.timedelta( days=k*6*7)
#         validation_end_date = x_training['date'].max() - datetime.timedelta( days=(k-1)*6*7)

#         # filtering dataset
#         training = x_training[x_training['date'] < validation_start_date]
#         validation = x_training[(x_training['date'] >= validation_start_date) & (x_training['date'] <= validation_end_date)]

#         # training and validation dataset
#         # training
#         xtraining = training.drop( ['date', 'sales'], axis=1 ) 
#         ytraining = training['sales']

#         # validation
#         xvalidation = validation.drop( ['date', 'sales'], axis=1 )
#         yvalidation = validation['sales']

#         # model
#         m = model.fit( xtraining, ytraining )

#         # prediction
#         yhat = m.predict( xvalidation )

#         # performance
#         m_result = ml_error( model_name, np.expm1( yvalidation ), np.expm1( yhat ) )

#         # store performance of each kfold iteration
#         mae_list.append(  m_result['MAE'] )
#         mape_list.append( m_result['MAPE'] )
#         rmse_list.append( m_result['RMSE'] )

#     return pd.DataFrame( {'Model Name': model_name,
#                           'MAE CV': np.round( np.mean( mae_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( mae_list ), 2 ).astype( str ),
#                           'MAPE CV': np.round( np.mean( mape_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( mape_list ), 2 ).astype( str ),
#                           'RMSE CV': np.round( np.mean( rmse_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( rmse_list ), 2 ).astype( str ) }, index=[0] )


# def mean_percentage_error( y, yhat ):
#     return np.mean( ( y - yhat ) / y )
     
    
# def mean_absolute_percentage_error( y, yhat ):
#     return np.mean( np.abs( ( y - yhat ) / y ) )

    
# def ml_error( model_name, y, yhat ):
#     mae = mean_absolute_error( y, yhat )
#     mape = mean_absolute_percentage_error( y, yhat )
#     rmse = np.sqrt( mean_squared_error( y, yhat ) )
    
#     return pd.DataFrame( { 'Model Name': model_name, 
#                            'MAE': mae, 
#                            'MAPE': mape,
#                            'RMSE': rmse }, index=[0] )

# def cramer_v( x, y ):
#     cm = pd.crosstab( x, y ).as_matrix()
#     n = cm.sum()
#     r, k = cm.shape
    
#     chi2 = ss.chi2_contingency( cm )[0]
#     chi2corr = max( 0, chi2 - (k-1)*(r-1)/(n-1) )
    
#     kcorr = k - (k-1)**2/(n-1)
#     rcorr = r - (r-1)**2/(n-1)
    
#     return np.sqrt( (chi2corr/n) / ( min( kcorr-1, rcorr-1 ) ) )



# def jupyter_settings():
#     %matplotlib inline
#     %pylab inline
    
#     plt.style.use( 'bmh' )
#     plt.rcParams['figure.figsize'] = [25, 12]
#     plt.rcParams['font.size'] = 24
    
#     display( HTML( '<style>.container { width:100% !important; }</style>') )
#     pd.options.display.max_columns = None
#     pd.options.display.max_rows = None
#     pd.set_option( 'display.expand_frame_repr', False )
    
#     sns.set()

In [ ]:
# jupyter_settings()

## 0.2 Load data

In [4]:
df_sales_raw = pd.read_csv( 'pmusachio/Downloads/repos/000-3_rossmann_store_sales_prediction/data/train.csv', low_memory=False )
df_store_raw = pd.read_csv( 'pmusachio/Downloads/repos/000-3_rossmann_store_sales_prediction/data/store.csv', low_memory=False )

# merge
df_raw = pd.merge( df_sales_raw, df_store_raw, how='left', on='Store' )

FileNotFoundError: [Errno 2] No such file or directory: 'pmusachio/Downloads/repos/000-3_rossmann_store_sales_prediction/data/train.csv'

# <font color = #7B68EE> 1.0 Data Description </font>

In [ ]:
df1 = df_raw.copy()

## 1.1 Columns Rename

In [ ]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 
            'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
            'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore( x )

cols_new = list( map( snakecase, cols_old ) )

# rename
df1.columns = cols_new

## 1.2 Data Dimensions

In [ ]:
print( 'Number of Rows: {}'.format( df1.shape[0] ) )
print( 'Number of Cols: {}'.format( df1.shape[1] ) )

## 1.3 Data Types

In [ ]:
df1['date'] = pd.to_datetime( df1['date'] )
df1.dtypes

## 1.4 Check NA

In [ ]:
df1.isna().sum()

## 1.5 Fillout/Replace NA

In [ ]:
df1.sample()

In [ ]:
#competition_distance        
df1['competition_distance'] = df1['competition_distance'].apply( lambda x: 200000.0 if math.isnan( x ) else x )

#competition_open_since_month
df1['competition_open_since_month'] = df1.apply( lambda x: x['date'].month if math.isnan( x['competition_open_since_month'] ) else x['competition_open_since_month'], axis=1 )

#competition_open_since_year 
df1['competition_open_since_year'] = df1.apply( lambda x: x['date'].year if math.isnan( x['competition_open_since_year'] ) else x['competition_open_since_year'], axis=1 )

#promo2_since_week           
df1['promo2_since_week'] = df1.apply( lambda x: x['date'].week if math.isnan( x['promo2_since_week'] ) else x['promo2_since_week'], axis=1 )

#promo2_since_year           
df1['promo2_since_year'] = df1.apply( lambda x: x['date'].year if math.isnan( x['promo2_since_year'] ) else x['promo2_since_year'], axis=1 )

#promo_interval              
month_map = {1: 'Jan',  2: 'Fev',  3: 'Mar',  4: 'Apr',  5: 'May',  6: 'Jun',  7: 'Jul',  8: 'Aug',  9: 'Sep',  10: 'Oct', 11: 'Nov', 12: 'Dec'}

df1['promo_interval'].fillna(0, inplace=True )

df1['month_map'] = df1['date'].dt.month.map( month_map )

df1['is_promo'] = df1[['promo_interval', 'month_map']].apply( lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split( ',' ) else 0, axis=1 )


In [ ]:
df1.isna().sum()

## 1.6 Change Data Types

In [ ]:
# competiton
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype( int )
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype( int )
    
# promo2
df1['promo2_since_week'] = df1['promo2_since_week'].astype( int )
df1['promo2_since_year'] = df1['promo2_since_year'].astype( int )

## 1.7 Descriptive Statistics

In [ ]:
num_attributes = df1.select_dtypes( include=['int64', 'float64'] )
cat_attributes = df1.select_dtypes( exclude=['int64', 'float64', 'datetime64[ns]'] )

### 1.7.1 Numerical Attributes

In [ ]:
# Central Tendency - mean, meadina 
ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame( num_attributes.apply( np.median ) ).T

# dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T 
d2 = pd.DataFrame( num_attributes.apply( min ) ).T 
d3 = pd.DataFrame( num_attributes.apply( max ) ).T 
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T 
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T 
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T 

# concatenar
m = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6] ).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

In [ ]:
sns.distplot( df1['competition_distance'], kde=False )

### 1.7.2 Categorical Attributes

In [ ]:
cat_attributes.apply( lambda x: x.unique().shape[0] )

In [ ]:
aux = df1[(df1['state_holiday'] != '0') & (df1['sales'] > 0)]

plt.subplot( 1, 3, 1 )
sns.boxplot( x='state_holiday', y='sales', data=aux )

plt.subplot( 1, 3, 2 )
sns.boxplot( x='store_type', y='sales', data=aux )

plt.subplot( 1, 3, 3 )
sns.boxplot( x='assortment', y='sales', data=aux )

# <font color = #7B68EE> 2.0 Feature Engineering </font>

In [ ]:
df2 = df1.copy()

## 2.1 Hypothesis Mindmap

In [ ]:
Image( 'img/MindMapHypothesis.png' )

## 2.2 Hypothesis Creation

| # | Store | Product | Time |
|---|-------|---------|------|
| 1 | Stores with a larger number of employees should sell more | Stores that invest more in Marketing should sell more | <font color = GREEN> **Stores open during the Christmas holiday should sell more** </font> |
| 2 | Stores with greater stock capacity should sell more | Stores with greater product display should sell more | <font color = GREEN> **Stores should sell more over the years** </font> |
| 3 | Larger stores should sell more | Stores with lower priced products should sell more | <font color = GREEN> **Stores should sell more in the second half of the year** </font> |
| 4 | <font color = GREEN> **Stores with larger assortments should sell more** </font> | Stores with more aggressive promotions (bigger discounts) should sell more | <font color = GREEN> **Stores should sell more after the 10th of each month** </font> |
| 5 | <font color = GREEN> **Stores with closer competitors should sell less** </font> | <font color = GREEN> **Stores with active promotions for longer should sell more** </font> | <font color = GREEN> **Stores should sell less on weekends** </font> |
| 6 | <font color = GREEN> **Stores with longer competitors should sell more** </font> | <font color = GREEN> **Stores with more promotion days should sell more** </font> | <font color = GREEN> **Stores should sell less during school holidays** </font> |
| 7 | | <font color = GREEN> **Stores with more consecutive promotions should sell more** </font> | |

## 2.3 Feature Creation

In [ ]:
# year
df2['year'] = df2['date'].dt.year

# month
df2['month'] = df2['date'].dt.month

# day
df2['day'] = df2['date'].dt.day

# week of year
df2['week_of_year'] = df2['date'].dt.weekofyear

# year week
df2['year_week'] = df2['date'].dt.strftime( '%Y-%W' )

# competition since
df2['competition_since'] = df2.apply( lambda x: datetime.datetime( year=x['competition_open_since_year'], month=x['competition_open_since_month'],day=1 ), axis=1 )
df2['competition_time_month'] = ( ( df2['date'] - df2['competition_since'] )/30 ).apply( lambda x: x.days ).astype( int )

# promo since
df2['promo_since'] = df2['promo2_since_year'].astype( str ) + '-' + df2['promo2_since_week'].astype( str )
df2['promo_since'] = df2['promo_since'].apply( lambda x: datetime.datetime.strptime( x + '-1', '%Y-%W-%w' ) - datetime.timedelta( days=7 ) )
df2['promo_time_week'] = ( ( df2['date'] - df2['promo_since'] )/7 ).apply( lambda x: x.days ).astype( int )

# assortment
df2['assortment'] = df2['assortment'].apply( lambda x: 'basic' if x == 'a' else 'extra' if x == 'b' else 'extended' )

# state holiday
df2['state_holiday'] = df2['state_holiday'].apply( lambda x: 'public_holiday' if x == 'a' else 'easter_holiday' if x == 'b' else 'christmas' if x == 'c' else 'regular_day' )

# <font color = #7B68EE> 3.0 Data Filtering </font>

In [ ]:
df3 = df2.copy()

## 3.1 Filtering Rows

In [ ]:
df3 = df3[(df3['open'] != 0) & (df3['sales'] > 0)]

## 3.2 Columns Selection

In [ ]:
cols_drop = ['customers', 'open', 'promo_interval', 'month_map']
df3 = df3.drop( cols_drop, axis=1 )

# <font color = #7B68EE> 4.0 Exploratoria Data Analysis </font>

In [ ]:
df4 = df3.copy()

## 4.1 Univariate Analysis

### 4.1.1 Response Variable

In [ ]:
sns.distplot( df4['sales'], kde=False  )

### 4.1.2 Numerical Variable

In [ ]:
num_attributes.hist( bins=25 );

### 4.1.3 Categorical Variable

In [ ]:
# state_holiday
plt.subplot( 3, 2, 1 )
a = df4[df4['state_holiday'] != 'regular_day']
sns.countplot( a['state_holiday'] )

plt.subplot( 3, 2, 2 )
sns.kdeplot( df4[df4['state_holiday'] == 'public_holiday']['sales'], label='public_holiday', shade=True )
sns.kdeplot( df4[df4['state_holiday'] == 'easter_holiday']['sales'], label='easter_holiday', shade=True )
sns.kdeplot( df4[df4['state_holiday'] == 'christmas']['sales'], label='christmas', shade=True )

# store_type
plt.subplot( 3, 2, 3 )
sns.countplot( df4['store_type'] )

plt.subplot( 3, 2, 4 )
sns.kdeplot( df4[df4['store_type'] == 'a']['sales'], label='a', shade=True )
sns.kdeplot( df4[df4['store_type'] == 'b']['sales'], label='b', shade=True )
sns.kdeplot( df4[df4['store_type'] == 'c']['sales'], label='c', shade=True )
sns.kdeplot( df4[df4['store_type'] == 'd']['sales'], label='d', shade=True )

# assortment
plt.subplot( 3, 2, 5 )
sns.countplot( df4['assortment'] )

plt.subplot( 3, 2, 6 )
sns.kdeplot( df4[df4['assortment'] == 'extended']['sales'], label='extended', shade=True )
sns.kdeplot( df4[df4['assortment'] == 'basic']['sales'], label='basic', shade=True )
sns.kdeplot( df4[df4['assortment'] == 'extra']['sales'], label='extra', shade=True )

## 4.2 Hypothesis Validation

In [ ]:
plt.suptitle("h01 - Stores with larger assortments should sell more"
             "Conclusion <font color = RED> FALSE </font> Stores with BIGGER ASSORTMENT sell LESS", y=1.02)

In [ ]:
plt.suptitle("h02 - Stores with closer competitors should sell less"
             "Conclusion <font color = RED> FALSE </font> Stores with CLOSER COMPETITORS sell MORE", y=1.02)

In [ ]:
plt.suptitle("h03 - Stores with longer competitors should sell more"
             "Conclusion <font color = RED> FALSE </font> Stores with LONGER COMPETITORS sell LESS", y=1.02)

In [ ]:
plt.suptitle("h04 - Stores with active promotions for longer should sell more"
             "Conclusion <font color = RED> FALSE </font> Stores with active promotions for longer sell less after a certain period of promotion", y=1.02)

In [ ]:
plt.suptitle("h06 - Stores with more consecutive promotions should sell more"
             "Conclusion <font color = RED> FALSE </font> Stores with more consecutive promotions sell less", y=1.02)

In [ ]:
plt.suptitle("h07 - Stores open during the Christmas holiday should sell more"
             "Conclusion <font color = RED> FALSE </font> Stores open during the Christmas holiday sell less", y=1.02)

In [ ]:
plt.suptitle("h08 - Stores should sell more over the years"
             "Conclusion <font color = RED> FALSE </font> Stores sell less over the years", y=1.02)

In [ ]:
plt.suptitle("h09 - Stores should sell more in the second half of the year"
             "Conclusion <font color = RED> FALSE </font> Stores sell less in the second half of the year", y=1.02)

In [ ]:
plt.suptitle("h10 - Stores should sell more after the 1"0th of each month"
             "Conclusion <font color = GREEN> **TRUE**" </font> Stores sell more after the 10th of each month", y=1.02)

In [ ]:
plt.suptitle("h11 - Stores should sell less on weekends"
             "Conclusion <font color = GREEN> **TRUE**" </font> Stores sell less on the weekend", y=1.02)

In [ ]:
plt.suptitle("h12 - Stores should sell less during scho"ol holidays"
             "Conclusion <font color = GREEN> **TRUE**" </font> Stores sell less during school holidays, except the months of July and August", y=1.02)

| #   | Hypothesis                                                | Conclusion                            | Comment |
|-----|-----------------------------------------------------------|---------------------------------------|---------|
| h01 | Stores with larger assortments should sell more           | <font color = RED> FALSE </font>      | Stores with BIGGER ASSORTMENT sell LESS |
| h02 | Stores with closer competitors should sell less           | <font color = RED> FALSE </font>      | Stores with CLOSER COMPETITORS sell MORE |
| h03 | Stores with longer competitors should sell more           | <font color = RED> FALSE </font>      | Stores with LONGER COMPETITORS sell LESS |
| h04 | Stores with active promotions for longer should sell more | <font color = RED> FALSE </font>      | Stores with active promotions for longer sell less after a certain period of promotion |
| h05 | ~~Stores with more promotion days should sell more~~      |                                       |  |
| h06 | Stores with more consecutive promotions should sell more  | <font color = RED> FALSE </font>      | Stores with more consecutive promotions sell less |
| h07 | Stores open during the Christmas holiday should sell more | <font color = RED> FALSE </font>      | Stores open during the Christmas holiday sell less |
| h08 | Stores should sell more over the years                    | <font color = RED> FALSE </font>      | Stores sell less over the years |
| h09 | Stores should sell more in the second half of the year    | <font color = RED> FALSE </font>      | Stores sell less in the second half of the year |
| h10 | Stores should sell more after the 10th of each month      | <font color = GREEN> **TRUE** </font> | Stores sell more after the 10th of each month |
| h11 | Stores should sell less on weekends                       | <font color = GREEN> **TRUE** </font> | Stores sell less on the weekend |
| h12 | Stores should sell less during school holidays            | <font color = GREEN> **TRUE** </font> | Stores sell less during school holidays, except the months of July and August |

## 4.3 Multivariate Analysis

### 4.3.1 Numerical Attributes

### 4.3.2 Categorical Attributes

# <font color = #7B68EE> 5.0 Data Preparation </font>

In [ ]:
df5 = df4.copy()

## 5.1 Standardization

## 5.2 Rescaling

## 5.3 Transformation

### 5.3.1 Encoding

### 5.3.2 Response Variable Transformation

### 5.3.3 Magnitude Transformation - Variável Resposta

### 5.3.4 Nature Transformation

## 5.4 Preparing Validation Data

# <font color = #7B68EE> 6.0 Feature Selection </font>

In [ ]:
df6 = df5.copy()

## 6.1 Split dataframe Training/Test

## 6.2 Feature Selector

### 6.2.1 Feature Selector Algorithm (boruta, xtra trees)

### 6.2.2 Feature Importance

## 6.3 Manual Feature Selection

# <font color = #7B68EE> 7.0 Machine Learning Modelling </font>

In [ ]:
df7 = df6.copy()

## 7.1 Data Preparation

## 7.2 Algorithms ML

## 7.2.1 Model 1, Treinamento, Previsao

## 7.2.2 Model 2, Treinamento, Previsao

## 7.2.3 Model 3, Treinamento, Previsao

## 7.3 Compare Model's Performance

### 7.3.1 Single Performance (W/O Cross-Validation)

### 7.3.2 Real Performance (W/ Cross-Validation)

## 7.4 Cumulative Gain

# <font color = #7B68EE> 8.0 Hyperparameter Fine-Tunning </font>

In [ ]:
df8 = df7.copy()

## 8.1 Usando o Optuna

## 8.2 Final Model (modelo, treinamento, previsao, performance)

# <font color = #7B68EE> 9.0 Performance Metrics </font>

In [ ]:
df9 = df8.copy()

- Cummulative Curve Manually
- Lift Curve Manually
- ROI Curve Manually

## 9.1 Business Performance

## 9.2 Total Performance

## 9.3 Machine Learning Performance

# <font color = #7B68EE> 10.0 Deploy to Production </font>

In [ ]:
df10 = df9.copy()

## 10.1 Project Class

## 10.2 API Handler

## 10.3 API Tester